In [ ]:
import pyjags
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# simulate from Be(2,5)
theta = np.random.beta(2,5,10000) # generate 10,000 values from Be(2,5)
print(theta[0:10]) # first ten simulated values
print(np.mean(theta)) # sample mean
print(np.percentile(theta,[2.5,97.5])) # 2.5% and 97.5% of empirical distn

In [ ]:
# simulate y from posterior predictive distribution
theta = np.random.beta(2,5,10000) # generate 10,000 values from Be(2,5)
print(theta[0:10]) # first ten simulated values
y = np.random.binomial(1,theta,10000) # generate 10,000 binary values with different probabilities
print(y[0:10]) #first ten values
print(np.bincount(y)/10000)  # frequency of 0 and 1

In [ ]:
beetles_code = '''
model {
mean_x = mean(x);
centered_x = x - mean_x;
alpha = alpha_star - beta*mean_x; ## original alpha
alpha_star ~ dnorm(0.0, 0.0001); ## prior for alpha_star
beta ~ dnorm(0.0, 0.0001); ## prior for beta
linpred = alpha_star + beta * centered_x;
for (i in 1:N)  {
    p[i] = ilogit(linpred[i]);
    yhat[i] = p[i]*n[i];  ## fitted values
    y[i] ~ dbin(p[i],n[i]) ## model for y
  }
}
'''

#An alternative would be to read the JAGS code from a separate file, as follows:
#
#with open("beetles.jag","r") as beetles_jags:
#    beetles_code = beetles_jags.read()

In [ ]:
beetles_x = np.array([1.6907, 1.7242, 1.7552, 1.7842, 1.8113, 1.8369, 1.8610, 1.8839])
beetles_n = np.array([59, 60, 62, 56, 63, 59, 62, 60])
beetles_y = np.array([6, 13, 18, 28, 52, 53, 61, 60])
beetles_N = 8

In [ ]:
beetles_model = pyjags.Model(beetles_code, data=dict(x=beetles_x, n=beetles_n, y = beetles_y, N=beetles_N), chains=4)
beetles_burnin = beetles_model.sample(500, vars=['alpha_star','beta']) #warmup/burn-in
beetles_samples = beetles_model.sample(2500, vars=['alpha_star','beta'])

In [ ]:
import rpy2
from rpy2.robjects.packages import importr
#If there are errors about missing R packages, run the code below:
#r_utils = importr("utils")
#r_utils.install_packages('coda')
r_coda = importr("coda")
from rpy2.robjects import pandas2ri
pandas2ri.activate()
#chain 1
beetles_chain1 = np.column_stack((beetles_samples['alpha_star'][0][:,0],beetles_samples['beta'][0][:,0]))
beetles_chain1_df = pd.DataFrame({'alpha_star':beetles_chain1[:,0],'beta':beetles_chain1[:,1]})
beetles_chain1_mcmc = r_coda.mcmc(pandas2ri.py2ri(beetles_chain1_df))
#chain 2
beetles_chain2 = np.column_stack((beetles_samples['alpha_star'][0][:,1],beetles_samples['beta'][0][:,1]))
beetles_chain2_df = pd.DataFrame({'alpha_star':beetles_chain2[:,0],'beta':beetles_chain2[:,1]})
beetles_chain2_mcmc = r_coda.mcmc(pandas2ri.py2ri(beetles_chain2_df))
#chain 3
beetles_chain3 = np.column_stack((beetles_samples['alpha_star'][0][:,2],beetles_samples['beta'][0][:,2]))
beetles_chain3_df = pd.DataFrame({'alpha_star':beetles_chain3[:,0],'beta':beetles_chain3[:,1]})
beetles_chain3_mcmc = r_coda.mcmc(pandas2ri.py2ri(beetles_chain3_df))
#chain 4
beetles_chain4 = np.column_stack((beetles_samples['alpha_star'][0][:,3],beetles_samples['beta'][0][:,3]))
beetles_chain4_df = pd.DataFrame({'alpha_star':beetles_chain4[:,0],'beta':beetles_chain4[:,1]})
beetles_chain4_mcmc = r_coda.mcmc(pandas2ri.py2ri(beetles_chain4_df))
#convert to mcmc_list object
beetles_chains=r_coda.mcmc_list(beetles_chain1_mcmc,beetles_chain2_mcmc,beetles_chain3_mcmc,beetles_chain4_mcmc)
#get n_eff and Rhat
beetles_n_eff = np.round(np.array(r_coda.effectiveSize(beetles_chains))) #round because must be an integer
beetles_rhat = np.array(r_coda.gelman_diag(beetles_chains).rx2("psrf"))
beetles_rhat = np.array([beetles_rhat[0][0],beetles_rhat[1][0]]) #extract point estimates
#calculate summary
beetles_alpha_star_summary = [np.mean(beetles_samples['alpha_star']),np.std(beetles_samples['alpha_star'])]
beetles_beta_summary = [np.mean(beetles_samples['beta']),np.std(beetles_samples['beta'])]
for i in [0.025,0.25,0.5,0.75,0.975]:
    beetles_alpha_star_summary.append(np.quantile(beetles_samples['alpha_star'],i))
    beetles_beta_summary.append(np.quantile(beetles_samples['beta'],i))
beetles_alpha_star_summary.append(beetles_n_eff[0])
beetles_alpha_star_summary.append(beetles_rhat[0])
beetles_beta_summary.append(beetles_n_eff[1])
beetles_beta_summary.append(beetles_rhat[1])
beetles_summary = pd.DataFrame([beetles_alpha_star_summary,beetles_beta_summary],columns=["mean","sd","2.5%","25%","50%","75%","97.5%","n_eff","Rhat"],index=["alpha_star","beta"])
beetles_summary.round(3)

In [ ]:
_ = plt.figure(figsize=(11,8.5))
_ = plt.plot(range(500),beetles_burnin['alpha_star'][0,:,0],color="red",label="Chain 1")
_ = plt.plot(range(500),beetles_burnin['alpha_star'][0,:,1],color="blue",label="Chain 2")
_ = plt.plot(range(500),beetles_burnin['alpha_star'][0,:,2],color="cyan",label="Chain 3")
_ = plt.plot(range(500),beetles_burnin['alpha_star'][0,:,3],color="green",label="Chain 4")
_ = plt.xlabel("iteration")
_ = plt.ylabel("alpha_star")
_ = plt.title("Traceplot for Beetles data: alpha_star")
_ = plt.legend()
_ = plt.show()

In [ ]:
_ = plt.figure(figsize=(11,8.5))
_ = plt.plot(range(501,3001),beetles_samples['alpha_star'][0,:,0],color="red",label="Chain 1")
_ = plt.plot(range(501,3001),beetles_samples['alpha_star'][0,:,1],color="blue",label="Chain 2")
_ = plt.plot(range(501,3001),beetles_samples['alpha_star'][0,:,2],color="cyan",label="Chain 3")
_ = plt.plot(range(501,3001),beetles_samples['alpha_star'][0,:,3],color="green",label="Chain 4")
_ = plt.xlabel("iteration")
_ = plt.ylabel("alpha_star")
_ = plt.title("Traceplot for Beetles data: alpha_star")
_ = plt.legend()
_ = plt.show()

In [ ]:
_ = plt.figure(figsize=(11,8.5))
_ = plt.plot(range(500),beetles_burnin['beta'][0,:,0],color="red",label="Chain 1")
_ = plt.plot(range(500),beetles_burnin['beta'][0,:,1],color="blue",label="Chain 2")
_ = plt.plot(range(500),beetles_burnin['beta'][0,:,2],color="cyan",label="Chain 3")
_ = plt.plot(range(500),beetles_burnin['beta'][0,:,3],color="green",label="Chain 4")
_ = plt.xlabel("iteration")
_ = plt.ylabel("beta")
_ = plt.title("Traceplot for Beetles data: beta")
_ = plt.legend()
_ = plt.show()

In [ ]:
_ = plt.figure(figsize=(11,8.5))
_ = plt.plot(range(501,3001),beetles_samples['beta'][0,:,0],color="red",label="Chain 1")
_ = plt.plot(range(501,3001),beetles_samples['beta'][0,:,1],color="blue",label="Chain 2")
_ = plt.plot(range(501,3001),beetles_samples['beta'][0,:,2],color="cyan",label="Chain 3")
_ = plt.plot(range(501,3001),beetles_samples['beta'][0,:,3],color="green",label="Chain 4")
_ = plt.xlabel("iteration")
_ = plt.ylabel("beta")
_ = plt.title("Traceplot for Beetles data: beta")
_ = plt.legend()
_ = plt.show()

In [ ]:
import pandas as pd
sleepstudy = pd.read_csv("sleepstudy.csv")

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
prev_subj = int(sleepstudy[['Subject']].iloc[0])
sleepstudy_id = [1]
for i in range(1,len(sleepstudy[['Subject']])):
    if int(sleepstudy[['Subject']].iloc[i])==prev_subj:
        sleepstudy_id.append(sleepstudy_id[-1])
    else:
        sleepstudy_id.append(sleepstudy_id[-1]+1)
    prev_subj = int(sleepstudy[['Subject']].iloc[i])

In [ ]:
sleepstudy_code = '''
model {
mean_x = mean(x);
for (i in 1:N){
    linpred[i] = alpha[id[i]] + beta[id[i]]*x[i];
}
sig2_inv ~ dgamma(0.001, 0.001);
sig = sqrt(1/sig2_inv);
tau2_alpha_inv ~ dgamma(0.001, 0.0001);
tau_alpha = sqrt(1/tau2_alpha_inv);
tau2_beta_inv ~ dgamma(0.001, 0.001);
tau_beta = sqrt(1/tau2_beta_inv);
for (j in 1:N_ID){
    alpha[j] ~ dnorm(300, tau2_alpha_inv);
    beta[j] ~ dnorm(10, tau2_beta_inv);
}
for (i in 1:N){
    y[i] ~ dnorm(linpred[i], sig2_inv);
    }
}
'''


In [ ]:
sleepstudy_model = pyjags.Model(sleepstudy_code, data=dict(N = len(sleepstudy_id), N_ID = max(sleepstudy_id), y = flatten(np.array(sleepstudy[['Reaction']])), id = sleepstudy_id, x = flatten(np.array(sleepstudy[['Days']])) ), chains=3)
_ = sleepstudy_model.sample(2000, vars = []) #warmup/burn-in
sleepstudy_samples = sleepstudy_model.sample(5000, vars = ["alpha","beta","linpred","y"])

In [ ]:
from collections import OrderedDict
#chain 1
sleepstudy_chain1 = np.column_stack((np.column_stack([sleepstudy_samples['alpha'][i][:,0] for i in range(18)]),np.column_stack([sleepstudy_samples['beta'][i][:,0] for i in range(18)])))
#the next line uses a dictionary comprehension and expression unpacking to merge dictionaries 
#see https://www.python.org/dev/peps/pep-0448/ as a reference for the latter
sleepstudy_chain1_df = pd.DataFrame(OrderedDict({**{'alpha_'+str(i+1):sleepstudy_chain1[:,i] for i in range(18)},**{'beta_'+str(i-17):sleepstudy_chain1[:,i] for i in range(18,36)}}))
sleepstudy_chain1_mcmc = r_coda.mcmc(pandas2ri.py2ri(sleepstudy_chain1_df))
#chain 2
sleepstudy_chain2 = np.column_stack((np.column_stack([sleepstudy_samples['alpha'][i][:,1] for i in range(18)]),np.column_stack([sleepstudy_samples['beta'][i][:,1] for i in range(18)])))
sleepstudy_chain2_df = pd.DataFrame(OrderedDict({**{'alpha_'+str(i+1):sleepstudy_chain2[:,i] for i in range(18)},**{'beta_'+str(i-17):sleepstudy_chain2[:,i] for i in range(18,36)}}))
sleepstudy_chain2_mcmc = r_coda.mcmc(pandas2ri.py2ri(sleepstudy_chain2_df))
#chain 3
sleepstudy_chain3 = np.column_stack((np.column_stack([sleepstudy_samples['alpha'][i][:,2] for i in range(18)]),np.column_stack([sleepstudy_samples['beta'][i][:,2] for i in range(18)])))
sleepstudy_chain3_df = pd.DataFrame(OrderedDict({**{'alpha_'+str(i+1):sleepstudy_chain3[:,i] for i in range(18)},**{'beta_'+str(i-17):sleepstudy_chain3[:,i] for i in range(18,36)}}))
sleepstudy_chain3_mcmc = r_coda.mcmc(pandas2ri.py2ri(sleepstudy_chain3_df))
#convert to mcmc_list object
sleepstudy_chains=r_coda.mcmc_list(sleepstudy_chain1_mcmc,sleepstudy_chain2_mcmc,sleepstudy_chain3_mcmc)
#get n_eff and Rhat
sleepstudy_n_eff = np.round(np.array(r_coda.effectiveSize(sleepstudy_chains))) #round because must be an integer
sleepstudy_rhat = np.array(r_coda.gelman_diag(sleepstudy_chains).rx2("psrf"))
sleepstudy_rhat = np.array([sleepstudy_rhat[i][0] for i in range(36)]) #extract point estimates
#calculate summary
sleepstudy_alpha_summary = [[np.mean(sleepstudy_samples['alpha'][i]),np.std(sleepstudy_samples['alpha'][i])] for i in range(18)]
sleepstudy_beta_summary = [[np.mean(sleepstudy_samples['beta'][i]),np.std(sleepstudy_samples['beta'][i])] for i in range(18)]
for i in [0.025,0.25,0.5,0.75,0.975]:
    for j in range(18):
        sleepstudy_alpha_summary[j].append(np.quantile(sleepstudy_samples['alpha'][j],i))
        sleepstudy_beta_summary[j].append(np.quantile(sleepstudy_samples['beta'][j],i))
for j in range(18):
    sleepstudy_alpha_summary[j].append(sleepstudy_n_eff[j])
    sleepstudy_alpha_summary[j].append(sleepstudy_rhat[j])
    sleepstudy_beta_summary[j].append(sleepstudy_n_eff[j+18])
    sleepstudy_beta_summary[j].append(sleepstudy_rhat[j+18])
sleepstudy_summary = pd.DataFrame(sleepstudy_alpha_summary+sleepstudy_beta_summary,columns=["mean","sd","2.5%","25%","50%","75%","97.5%","n_eff","Rhat"],index=['alpha_'+str(i+1) for i in range(18)]+['beta_'+str(i+1) for i in range(18)])
sleepstudy_summary.round(3)

In [ ]:
ysleep = np.array(flatten(np.array(sleepstudy[['Reaction']])))
ysleep_pred = np.mean(np.mean(sleepstudy_samples['linpred'],axis=1),axis=1)
_ = plt.figure(figsize=(11,8.5))
_ = plt.scatter(ysleep,ysleep_pred)
_ = plt.title("Observed and fitted reaction times from Bayesian hierarchical linear model")
_ = plt.xlabel("Observed reaction time (ms)")
_ = plt.ylabel("Predicted reaction time (ms)")
minval = min(np.min(ysleep),np.min(ysleep_pred))
maxval = max(np.max(ysleep),np.max(ysleep_pred))
_ = plt.plot([minval,maxval],[minval,maxval],color="black")
_ = plt.show()

In [ ]:
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import gridspec
i = 0
plt.figure(figsize=(11,8.5))
gs  = gridspec.GridSpec(3, 6)
gs.update(wspace=0.5, hspace=0.5)
for subj in np.unique(sleepstudy['Subject']):
    ss_extract = sleepstudy.loc[sleepstudy['Subject']==subj]
    ss_extract_ols = sm.ols(formula="Reaction~Days",data=ss_extract).fit() #if we want to analyze fit later
    #new subplot
    plt.subplot(gs[i])
    #plot without confidence intervals
    sns.regplot(x='Days', y='Reaction', ci=None, data=ss_extract).set_title('Subject '+str(subj))
    if i not in [0,6,12]:
        plt.ylabel("")
    i+=1
_ = plt.figlegend(['per-subject'],loc = 'lower center', ncol=6)
_ = plt.show()

In [ ]:
i = 0
plt.figure(figsize=(11,8.5))
for subj in np.unique(sleepstudy['Subject']):
    ss_extract = sleepstudy.loc[sleepstudy['Subject']==subj]
    #new subplot
    plt.subplot(gs[i])
    #plot without confidence intervals
    sns.regplot(x='Days', y='Reaction', ci=None, data=ss_extract).set_title('Subject '+str(subj)) 
    sns.regplot(x='Days', y='Reaction', ci=None, scatter=False, data=sleepstudy)
    if i not in [0,6,12]:
        plt.ylabel("")
    i+=1
_ = plt.figlegend(['per-subject','pooled'],loc = 'lower center', ncol=6)
_ = plt.show()

In [ ]:
i = 0
plt.figure(figsize=(11,8.5))
subj_arr = np.unique(sleepstudy['Subject'])
for subj in subj_arr:
    ss_extract = sleepstudy.loc[sleepstudy['Subject']==subj]
    #new subplot
    plt.subplot(gs[i])
    #plot without confidence intervals
    sns.regplot(x='Days', y='Reaction', ci=None, data=ss_extract).set_title('Subject '+str(subj)) 
    sns.regplot(x='Days', y='Reaction', ci=None, scatter=False, data=sleepstudy)
    subj_num = int(np.where(subj_arr==subj)[0])
    hmodel_fit = [np.mean(sleepstudy_samples['alpha'][subj_num])+np.mean(sleepstudy_samples['beta'][subj_num])*x for x in range(-1,11)]
    sns.lineplot(x=range(-1,11),y=hmodel_fit)
    if i not in [0,6,12]:
        plt.ylabel("")
    i+=1
_ = plt.figlegend(['per-subject','pooled','hierarchical'],loc = 'lower center', ncol=6)
_ = plt.show()

In [ ]:
books = {}
def readbook(booknum):
    with open("harrypotter_book"+str(booknum)+".txt", "r") as book: 
        books[booknum] = book.read()

In [ ]:
for i in range(1,8):
    readbook(i)

In [ ]:
import re
for i in books:
    books[i] = re.split("Chapter\s?\d*|Epilogue",books[i]) #split each book into chapters
    #note that coming up with this regular expression required some trial and error (e.g. finding the epilogue)

In [ ]:
#references on LDA in Python: 
#https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
#import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.update(['harry','hermione','ron']) #add stop words for three main characters
stop_words.update(['said','got','get','would','could']) #empirically find these words very common
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[\w\']+')

In [ ]:
for book in books:
    for i in range(len(books[book])):
        #make everything lower case
        books[book][i] = books[book][i].lower()
        #tokenize
        books[book][i] = tokenizer.tokenize(books[book][i])
        #remove stop words
        books[book][i] = [j for j in books[book][i] if j not in stop_words]

In [ ]:
from gensim.corpora import Dictionary

In [ ]:
#concatenate chapters over all books
masterbooks = []
for book in books:
    for chapter in books[book]:
        masterbooks.append(chapter)
masterdictionary = {}
mastercorpus = {}
masterdictionary = Dictionary(masterbooks)
mastercorpus = [masterdictionary.doc2bow(chapter) for chapter in masterbooks]

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()
X = countvec.fit_transform(flatten(masterbooks))
Xsum = X.sum(axis=0)
words_freq = [(word, Xsum[0, idx]) for word, idx in countvec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[:20]

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

In [ ]:
#finding optimal number of topics via coherence measure
coherence_vals = []
for ntop in range(1,31):
   mod = LdaModel(mastercorpus, num_topics = ntop, id2word = masterdictionary, passes=10)
   cmod = CoherenceModel(model=mod, corpus=mastercorpus, dictionary=masterdictionary, coherence='u_mass')
   cval = cmod.get_coherence()
   print(ntop,cval)
   coherence_vals.append(cval)

In [ ]:
_ = plt.figure(figsize=(11,8.5))
_ = plt.plot(range(1,31),coherence_vals)
_ = plt.xlabel("Number of Topics")
_ = plt.ylabel("Coherence Score")
_ = plt.show()

In [ ]:
masterldamodel = LdaModel(mastercorpus, num_topics=16, id2word = masterdictionary, passes=10)

In [ ]:
top_words = [[word for word,_ in masterldamodel.show_topic(topicno, topn=50)] for topicno in range(masterldamodel.num_topics)]
top_betas = [[beta for _,beta in masterldamodel.show_topic(topicno, topn=50)] for topicno in range(masterldamodel.num_topics)]
print("Top Topics:")
for topicno, words in enumerate(top_words):
    print("%i: %s" % (topicno, ' '.join(words[:15])))
print("Top Topic Betas:")
for topicno, betas in enumerate(top_betas):
    print("%i: %s" % (topicno, ' '.join(map(str,betas[:15]))))

In [ ]:
top_words[0][:5]
top_betas[0][:5]
gs  = gridspec.GridSpec(2,3)
gs.update(wspace=0.5, hspace=0.5)
plt.figure(figsize=(11,8.5))
for i in range(6):
    #new subplot
    ax = plt.subplot(gs[i])
    plt.barh(range(5), top_betas[i][:5], align='center',color='blue', ecolor='black')
    ax.invert_yaxis()
    ax.set_yticks(range(5))
    ax.set_yticklabels(top_words[i][:5])
    plt.title("Topic "+str(i))
_ = plt.show()

In [ ]:
theta = masterldamodel.inference(mastercorpus)[0]
#note that dimensions of theta are number of documents (here, chapters) x number of topics in LDA model
for row in range(len(theta)): #normalize rows
    theta[row] = theta[row]/sum(theta[row])
theta_df = pd.DataFrame(theta)
#first column is document number (chapter over all Harry Potter books), second column is topic number
theta_df.stack().nlargest(10)

In [ ]:
#pip install pyldavis (if not already installed)
# note that the dynamic visualizations typically require an internet connection in order to work
import pyLDAvis
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()

In [ ]:
mastervis = pyLDAvis.gensim.prepare(masterldamodel, mastercorpus, masterdictionary)
mastervis

In [ ]:
#appendix:
#separate LDA models for each book
dictionary = {}
corpus = {}
for book in books:
    dictionary[book] = Dictionary(books[book])
    corpus[book] = [dictionary[book].doc2bow(chapter) for chapter in books[book]]
ldamodel = {}
for i in books:
    ldamodel[i] = LdaModel(corpus[i], num_topics=16, id2word = dictionary[i], passes=10)

In [ ]:
visbook1 = pyLDAvis.gensim.prepare(ldamodel[1], corpus[1], dictionary[1])
visbook1